<a href="https://colab.research.google.com/github/jhuarancca/MachineLearningEngineerI/blob/main/Telco_Customer_Churn_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
blastchar_telco_customer_churn_path = kagglehub.dataset_download('blastchar/telco-customer-churn')

print('Data source import complete.')


# Telco Customer Churn Analysis

## Problem

* It is requested to develop a machine learning model to predict customers who are likely to leave the company.

* Before building the model, it is expected that the necessary data analysis and feature engineering steps are completed.

## Dataset Information:

* The Telco Customer Churn dataset contains information about 7,043 customers who received landline phone and internet services from a fictional telecommunications company operating in California during the third quarter.
* It includes information on which customers stayed, left, or signed up for the service.
* 21 Variables and 7,043 Observations

## Features

**CustomerID:** Customer ID

**Gender:** Customer's gender

**SeniorCitizen:** Whether the customer is a senior citizen (1, 0)

**Partner:** Whether the customer has a partner (Yes, No) — indicates marital status

**Dependents:** Whether the customer has dependents (Yes, No) — such as children, parents, or grandparents

**Tenure:** Number of months the customer has stayed with the company

**PhoneService:** Whether the customer has phone service (Yes, No)

**MultipleLines:** Whether the customer has multiple lines (Yes, No, No phone service)

**InternetService:** Customer’s internet service provider (DSL, Fiber optic, No)

**OnlineSecurity:** Whether the customer has online security service (Yes, No, No internet service)

**OnlineBackup:** Whether the customer has online backup service (Yes, No, No internet service)

**DeviceProtection:** Whether the customer has device protection service (Yes, No, No internet service)

**TechSupport:** Whether the customer has technical support service (Yes, No, No internet service)

**StreamingTV:** Whether the customer is streaming TV (Yes, No, No internet service) — indicates if they use their internet service to stream TV shows through a third-party provider

**StreamingMovies:** Whether the customer is streaming movies (Yes, No, No internet service) — indicates if they use their internet service to stream movies through a third-party provider

**Contract:** Customer's contract term (Month-to-month, One year, Two year)

**PaperlessBilling:** Whether the customer has paperless billing (Yes, No)

**PaymentMethod:** Customer's payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))

**MonthlyCharges:** The amount charged to the customer monthly

**TotalCharges:** The total amount charged to the customer

**Churn:** Whether the customer churned (Yes or No) — customers who left within the last month or quarter

## Additional Notes:
* Each row represents a unique customer.

* Variables include information about customer services, account information, and demographic data.

**Customer services:** phone, multiple lines, internet, online security, online backup, device protection, tech support, streaming TV, and streaming movies.

**Customer account information:** tenure, contract, payment method, paperless billing, monthly charges, and total charges.

**Customer demographics:** gender, age range, and whether they have a partner or dependents.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pprint import pprint
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import warnings
warnings.simplefilter(action="ignore")


pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

## 1) Exploratory data analysis (EDA)

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

In [ ]:
 # We need to change the type of the variables TotalCharges and churn

df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors='coerce')
df["Churn"] = df["Churn"].apply(lambda x : 1 if x == "Yes" else 0)

df.info()

### 1.1 Determination of Numeric And Categorical Variables

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
    Returns the names of categorical, numerical, and categorical but cardinal variables in the dataset.
    Note: Numerical-looking categorical variables are also included in the categorical variables.

    Parameters
    ------
        dataframe: dataframe
                The dataframe from which the variable names are to be retrieved
        cat_th: int, optional
                Class threshold value for numerical but categorical variables
        car_th: int, optional
                Class threshold value for categorical but cardinal variables

    Returns
    ------
        cat_cols: list
                List of categorical variables
        num_cols: list
                List of numerical variables
        cat_but_car: list
                List of categorical-looking but cardinal variables

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))

    Notes
    ------
        cat_cols + num_cols + cat_but_car = total number of variables
        num_but_cat is included in cat_cols.
    """
    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    return cat_cols, num_cols, cat_but_car

cat_cols, num_cols, cat_but_car = grab_col_names(df)

print("************ Categorical Variables ************  ")
pprint(cat_cols)
print("************ Numerical Variables ************")
pprint(num_cols)
print("************ Categorical but Cardinal Variables ************")
pprint(cat_but_car)

### 1.2 ANALYSIS OF CATEGORICAL VARIABLES

In [ ]:
def cat_summary(dataframe, col_name, plot=True):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()

for col in cat_cols:
    cat_summary(df, col)

### 1.3 ANALYSIS OF NUMERİCAL VARIABLES

In [ ]:
def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)

    if plot:
        dataframe[numerical_col].hist(bins=20)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show()

for col in num_cols:
    num_summary(df, col, plot=True)

### 1.4 ANALYSIS OF NUMERICAL VARIABLES BY TARGET

In [ ]:
def target_summary_with_num(dataframe, target, numerical_col):
    print(dataframe.groupby(target).agg({numerical_col: "mean"}), end="\n\n\n")

for col in num_cols:
    target_summary_with_num(df, "Churn", col)

# As Tenure increases, Churn Risk decreases.
# Churning customers have more MonthlyCharge. In other words, the higher the bill, the higher the probability of Churn.
# Loyal customers paid more. In other words, we can say that customers who pay more are more loyal customers.

### 1.5 ANALYSIS OF CATEGORICAL VARIABLES BY TARGET

In [ ]:
def target_summary_with_cat(dataframe, target, categorical_col):
    print(categorical_col)
    print(pd.DataFrame({"TARGET_MEAN": dataframe.groupby(categorical_col)[target].mean(),
                        "Count": dataframe[categorical_col].value_counts(),
                        "Ratio": 100 * dataframe[categorical_col].value_counts() / len(dataframe)}), end="\n\n\n")

for col in cat_cols:
    target_summary_with_cat(df, "Churn", col)

plt.figure(figsize=(15, 5))

### 1.6 CORRELATION

In [ ]:
df[num_cols].corr() # TotalCharges & tenure correlation is 0.825

f, ax = plt.subplots(figsize=[18, 13])
sns.heatmap(df[num_cols].corr(), annot=True, fmt=".2f", ax=ax, cmap="magma")
ax.set_title("Correlation Matrix", fontsize=20)
plt.show()

df[num_cols].corrwith(df["Churn"]).sort_values(ascending=False)

## 2. FEATURE ENGINEERING

### 2.1 MISSING VALUE ANALYSİS

In [ ]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")
    if na_name:
        return na_columns

na_columns = missing_values_table(df, na_name=True)
df["TotalCharges"].fillna(df["TotalCharges"].median(), inplace=True)

In [ ]:
df.isnull().sum()

### BASE MODEL

In [ ]:
dff = df.copy()
cat_cols = [col for col in cat_cols if col not in ["Churn"]]

def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe
dff = one_hot_encoder(dff, cat_cols, drop_first=True)

y = dff["Churn"]
X = dff.drop(["Churn","customerID"], axis=1)

In [ ]:
# All models

models = [('LR', LogisticRegression(random_state=12345)),
          ('KNN', KNeighborsClassifier()),
          ('CART', DecisionTreeClassifier(random_state=12345)),
          ('RF', RandomForestClassifier(random_state=12345)),
          ('SVM', SVC(gamma='auto', random_state=12345)),
          ('XGB', XGBClassifier(random_state=12345)),
          ("LightGBM", LGBMClassifier(random_state=12345, verbose = -1)),
          ("CatBoost", CatBoostClassifier(verbose=False, random_state=12345))]


for name, model in models:
    cv_results = cross_validate(model, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc", "precision", "recall"])
    print(f"########## {name} ##########")
    print(f"Accuracy: {round(cv_results['test_accuracy'].mean(), 4)}")
    print(f"Auc: {round(cv_results['test_roc_auc'].mean(), 4)}")
    print(f"Recall: {round(cv_results['test_recall'].mean(), 4)}")
    print(f"Precision: {round(cv_results['test_precision'].mean(), 4)}")
    print(f"F1: {round(cv_results['test_f1'].mean(), 4)}")

### 2.2 OUTLIERS DETECTION & IMPUTATION

In [ ]:
def outlier_thresholds(dataframe, col_name, q1=0.05, q3=0.95):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

def replace_with_thresholds(dataframe, variable, q1=0.05, q3=0.95):
    low_limit, up_limit = outlier_thresholds(dataframe, variable, q1=0.05, q3=0.95)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

for col in num_cols:
    print(col, check_outlier(df, col))
    if check_outlier(df, col):
        replace_with_thresholds(df, col)


### 2.3 FEATURE ENGINEERING

In [ ]:
# Creating a yearly categorical variable from the "tenure" feature
df.loc[(df["tenure"] >= 0) & (df["tenure"] <= 12), "NEW_TENURE_YEAR"] = "0-1 Year"
df.loc[(df["tenure"] > 12) & (df["tenure"] <= 24), "NEW_TENURE_YEAR"] = "1-2 Year"
df.loc[(df["tenure"] > 24) & (df["tenure"] <= 36), "NEW_TENURE_YEAR"] = "2-3 Year"
df.loc[(df["tenure"] > 36) & (df["tenure"] <= 48), "NEW_TENURE_YEAR"] = "3-4 Year"
df.loc[(df["tenure"] > 48) & (df["tenure"] <= 60), "NEW_TENURE_YEAR"] = "4-5 Year"
df.loc[(df["tenure"] > 60) & (df["tenure"] <= 72), "NEW_TENURE_YEAR"] = "5-6 Year"

# Labeling customers with 1 or 2-year contracts as "Engaged"
df["NEW_Engaged"] = df["Contract"].apply(lambda x: 1 if x in ["One year", "Two year"] else 0)

# Customers who do not have any backup, protection, or support services
df["NEW_noProt"] = df.apply(lambda x: 1 if (x["OnlineBackup"] != "Yes") or (x["DeviceProtection"] != "Yes") or (x["TechSupport"] != "Yes") else 0, axis=1)

# Customers with a monthly contract and who are young
df["NEW_Young_Not_Engaged"] = df.apply(lambda x: 1 if (x["NEW_Engaged"] == 0) and (x["SeniorCitizen"] == 0) else 0, axis=1)

# The total number of services received by the customer
df['NEW_TotalServices'] = (df[['PhoneService', 'InternetService', 'OnlineSecurity',
                               'OnlineBackup', 'DeviceProtection', 'TechSupport',
                               'StreamingTV', 'StreamingMovies']] == 'Yes').sum(axis=1)

# Customers who subscribe to any streaming service
df["NEW_FLAG_ANY_STREAMING"] = df.apply(lambda x: 1 if (x["StreamingTV"] == "Yes") or (x["StreamingMovies"] == "Yes") else 0, axis=1)

# Does the customer make automatic payments?
df["NEW_FLAG_AutoPayment"] = df["PaymentMethod"].apply(lambda x: 1 if x in ["Bank transfer (automatic)", "Credit card (automatic)"] else 0)

# Average monthly payment
df["NEW_AVG_Charges"] = df["TotalCharges"] / (df["tenure"] + 1)

# Increase of the current price compared to the average price
df["NEW_Increase"] = df["NEW_AVG_Charges"] / df["MonthlyCharges"]

# Fee per service
df["NEW_AVG_Service_Fee"] = df["MonthlyCharges"] / (df['NEW_TotalServices'] + 1)


### 2.4 ENCODING

In [ ]:
# Separating variables based on their data types
cat_cols, num_cols, cat_but_car = grab_col_names(df)

# LABEL ENCODING
def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

binary_cols = [col for col in df.columns if df[col].dtypes == "O" and df[col].nunique() == 2]

for col in binary_cols:
    df = label_encoder(df, col)

# One-Hot Encoding Process
# Updating the cat_cols list
cat_cols = [col for col in cat_cols if col not in binary_cols and col not in ["Churn", "NEW_TotalServices"]]

def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

df = one_hot_encoder(df, cat_cols, drop_first=True)

df.head()

## ALL MODELS WITH NEW FEATURES

In [ ]:
y = df["Churn"]
X = df.drop(["Churn", "customerID"], axis=1)

##################################
# PERFORMANCE ANALYSIS WITH BASIC MODELS
##################################

models = [
    ('LR', LogisticRegression(random_state=12345)),
    ('KNN', KNeighborsClassifier()),
    ('CART', DecisionTreeClassifier(random_state=12345)),
    ('RF', RandomForestClassifier(random_state=12345)),
    ('SVM', SVC(gamma='auto', random_state=12345)),
    ('XGB', XGBClassifier(random_state=12345)),
    ('LightGBM', LGBMClassifier(random_state=12345, verbose = -1)),
    ('CatBoost', CatBoostClassifier(verbose=False, random_state=12345))
]

for name, model in models:
    cv_results = cross_validate(model, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc", "precision", "recall"])
    print(f"\n########## {name} ##########")
    print(f"Accuracy: {round(cv_results['test_accuracy'].mean(), 4)}")
    print(f"AUC: {round(cv_results['test_roc_auc'].mean(), 4)}")
    print(f"Recall: {round(cv_results['test_recall'].mean(), 4)}")
    print(f"Precision: {round(cv_results['test_precision'].mean(), 4)}")
    print(f"F1 Score: {round(cv_results['test_f1'].mean(), 4)}")


In [ ]:
##################################
# DEFINING HYPERPARAMETER RANGES
##################################

models_params = {
    "RandomForest": (
        RandomForestClassifier(random_state=17),
        {
            "max_depth": np.arange(3, 20),
            "max_features": [3, 5, 7, "auto", "sqrt"],
            "min_samples_split": np.arange(2, 20),
            "n_estimators": np.arange(100, 1000, 50)
        }
    ),
    "XGBoost": (
        XGBClassifier(random_state=17, verbosity=0),
        {
            "learning_rate": [0.1, 0.01, 0.001],
            "max_depth": np.arange(3, 20),
            "n_estimators": np.arange(100, 1000, 50),
            "colsample_bytree": [0.5, 0.7, 1]
        }
    ),
    "LightGBM": (
        LGBMClassifier(random_state=17, verbose = -1),
        {
            "learning_rate": [0.1, 0.01, 0.001],
            "n_estimators": np.arange(100, 1000, 100),
            "colsample_bytree": [0.5, 0.7, 1]
        }
    ),
    "CatBoost": (
        CatBoostClassifier(random_state=17, verbose=False),
        {
            "iterations": [200, 500, 700],
            "learning_rate": [0.1, 0.01],
            "depth": [3, 4, 5, 6]
        }
    )
}

In [ ]:
##################################
# RANDOMIZED + GRID SEARCH FUNCTION
##################################

def random_and_grid_search(models_params, X, y, random_iter=20, cv_random=3, cv_grid=5, plot_feature_importance=False):
    results = []
    best_params_all = {}
    final_models = {}

    for name, (model, random_param_grid) in models_params.items():
        print(f"\n########## {name} Model Training Started ##########")

        # 1. RandomizedSearchCV stage
        random_search = RandomizedSearchCV(
            model,
            random_param_grid,
            n_iter=random_iter,
            cv=cv_random,
            n_jobs=-1,
            random_state=17,
            verbose=0
        ).fit(X, y)

        print(f"RandomizedSearch Best Params for {name}: {random_search.best_params_}")

        # 2. Preparing parameters for GridSearchCV
        best_params = random_search.best_params_
        grid_param_grid = {}

        for param_name, best_val in best_params.items():
            if isinstance(best_val, (int, float)):
                if param_name in ["learning_rate", "colsample_bytree", "feature_fraction"]:
                    grid_param_grid[param_name] = [
                        max(0.0001, best_val * 0.9),
                        best_val,
                        min(1.0, best_val * 1.1)
                    ]
                else:
                    grid_param_grid[param_name] = [
                        max(1, best_val - 1),
                        best_val,
                        best_val + 1
                    ]
            else:
                grid_param_grid[param_name] = [best_val]

        # 3. GridSearchCV stage
        grid_search = GridSearchCV(
            model,
            grid_param_grid,
            cv=cv_grid,
            n_jobs=-1,
            verbose=0
        ).fit(X, y)

        print(f"GridSearch Best Params for {name}: {grid_search.best_params_}")

        best_params_all[name] = grid_search.best_params_

        # 4. Training the final model with best parameters
        final_model = model.set_params(**grid_search.best_params_, random_state=17).fit(X, y)
        final_models[name] = final_model

        # 5. Cross-validation performance measurement
        cv_results = cross_validate(final_model, X, y, cv=10, scoring=["accuracy", "f1", "roc_auc"])

        results.append({
            "Model": name,
            "Accuracy": round(cv_results["test_accuracy"].mean(), 4),
            "F1 Score": round(cv_results["test_f1"].mean(), 4),
            "ROC AUC": round(cv_results["test_roc_auc"].mean(), 4)
        })

    # 6. Convert results into a DataFrame
    results_df = pd.DataFrame(results).sort_values(by="ROC AUC", ascending=False).reset_index(drop=True)

    # 7. Feature Importance plotting (optional)
    if plot_feature_importance:
        for name, model in final_models.items():
            if hasattr(model, "feature_importances_"):
                importances = model.feature_importances_
                feature_names = X.columns
                feature_imp_df = pd.DataFrame({"Feature": feature_names, "Importance": importances})
                feature_imp_df = feature_imp_df.sort_values(by="Importance", ascending=False)

                plt.figure(figsize=(10, 6))
                plt.barh(feature_imp_df["Feature"], feature_imp_df["Importance"], color="royalblue")
                plt.title(f"{name} - Feature Importances", fontsize=16, fontweight="bold")
                plt.gca().invert_yaxis()
                plt.grid(True, axis='x', linestyle='--', alpha=0.7)
                plt.tight_layout()
                plt.show()
            else:
                print(f"\nNo 'feature_importances_' attribute found for {name} model. (Skipped.)")

    return results_df, best_params_all


In [ ]:
##################################
# FUNCTION EXECUTION
##################################

results_df, best_params_all = random_and_grid_search(models_params, X, y, plot_feature_importance=True)

# Results
print("\n########## Final Model Performances ##########")
print(results_df)

print("\n########## Best Hyperparameters ##########")
print(best_params_all)